In [ ]:
import cv2, dlib, numpy as np
from collections import OrderedDict
from google.colab.patches import cv2_imshow
from PIL import Image, ImageEnhance
import os
import os.path
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp "/content/gdrive/MyDrive/shape_predictor_68_face_landmarks.dat" "/content/shape_predictor_68_face_landmarks.dat"

In [ ]:
if not os.path.exists('/content/unprocessed_train_data1'):
    os.mkdir('/content/unprocessed_train_data1')

if not os.path.exists('/content/processed_train_data1'):
    os.mkdir('/content/processed_train_data1')

if not os.path.exists('/content/processed_train_data1/real'):
    os.mkdir('/content/processed_train_data1/real')

if not os.path.exists('/content/processed_train_data1/fake'):
    os.mkdir('/content/processed_train_data1/fake')

In [ ]:
!unzip '/content/gdrive/MyDrive/unprocessed_trainset_1.zip' -d '/content/unprocessed_train_data1'

In [ ]:
def rect_to_bb(rect):
	# rectobject to (x,y,w,h)
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
	return (x, y, w, h)

In [ ]:
def shape_to_np(shape, dtype="int"):
	coords = np.zeros((68, 2), dtype=dtype)
	for i in range(0, 68):
		coords[i] = (shape.part(i).x, shape.part(i).y)
	# 68 landmarks to np array
	return coords

In [ ]:
def umeyama( src, dst, estimate_scale ):
    """Estimate N-D similarity transformation with or without scaling.
    Parameters
    ----------
    src : (M, N) array
        Source coordinates.
    dst : (M, N) array
        Destination coordinates.
    estimate_scale : bool
        Whether to estimate scaling factor.
    Returns
    -------
    T : (N + 1, N + 1)
        The homogeneous similarity transformation matrix. The matrix contains
        NaN values only if the problem is not well-conditioned.
    References
    ----------
    .. [1] "Least-squares estimation of transformation parameters between two
            point patterns", Shinji Umeyama, PAMI 1991, DOI: 10.1109/34.88573
    """

    num = src.shape[0]
    dim = src.shape[1]

    # Compute mean of src and dst.
    src_mean = src.mean(axis=0)
    dst_mean = dst.mean(axis=0)

    # Subtract mean from src and dst.
    src_demean = src - src_mean
    dst_demean = dst - dst_mean

    # Eq. (38).
    A = np.dot(dst_demean.T, src_demean) / num

    # Eq. (39).
    d = np.ones((dim,), dtype=np.double)
    if np.linalg.det(A) < 0:
        d[dim - 1] = -1

    T = np.eye(dim + 1, dtype=np.double)

    U, S, V = np.linalg.svd(A)

    # Eq. (40) and (43).
    rank = np.linalg.matrix_rank(A)
    if rank == 0:
        return np.nan * T
    elif rank == dim - 1:
        if np.linalg.det(U) * np.linalg.det(V) > 0:
            T[:dim, :dim] = np.dot(U, V)
        else:
            s = d[dim - 1]
            d[dim - 1] = -1
            T[:dim, :dim] = np.dot(U, np.dot(np.diag(d), V))
            d[dim - 1] = s
    else:
        T[:dim, :dim] = np.dot(U, np.dot(np.diag(d), V.T))

    if estimate_scale:
        # Eq. (41) and (42).
        scale = 1.0 / src_demean.var(axis=0).sum() * np.dot(S, d)
    else:
        scale = 1.0

    T[:dim, dim] = dst_mean - scale * np.dot(T[:dim, :dim], src_mean.T)
    T[:dim, :dim] *= scale

    return T

In [ ]:
# mean coords of landmarks on alligned face
#used to align face when simulating deepfakes pipeline
# taken from yuezen's research paper implementation
y = np.array([
    0.106454, 0.038915, 0.0187482, 0.0344891, 0.0773906, 0.0773906, 0.0344891,
    0.0187482, 0.038915, 0.106454, 0.203352, 0.307009, 0.409805, 0.515625, 0.587326,
    0.609345, 0.628106, 0.609345, 0.587326, 0.216423, 0.178758, 0.179852, 0.231733,
    0.245099, 0.244077, 0.231733, 0.179852, 0.178758, 0.216423, 0.244077, 0.245099,
    0.780233, 0.745405, 0.727388, 0.742578, 0.727388, 0.745405, 0.780233, 0.864805,
    0.902192, 0.909281, 0.902192, 0.864805, 0.784792, 0.778746, 0.785343, 0.778746,
    0.784792, 0.824182, 0.831803, 0.824182])
x = np.array([
    0.000213256, 0.0752622, 0.18113, 0.29077, 0.393397, 0.586856, 0.689483, 0.799124,
    0.904991, 0.98004, 0.490127, 0.490127, 0.490127, 0.490127, 0.36688, 0.426036,
    0.490127, 0.554217, 0.613373, 0.121737, 0.187122, 0.265825, 0.334606, 0.260918,
    0.182743, 0.645647, 0.714428, 0.793132, 0.858516, 0.79751, 0.719335, 0.254149,
    0.340985, 0.428858, 0.490127, 0.551395, 0.639268, 0.726104, 0.642159, 0.556721,
    0.490127, 0.423532, 0.338094, 0.290379, 0.428096, 0.490127, 0.552157, 0.689874,
    0.553364, 0.490127, 0.42689])
aligned_landmarks_norm = np.stack([x, y], axis=1)

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Main loop

In [ ]:
negImages=[]
positiveImages=[]
invalidNames=[]
validNames=[]
path = ('/content/unprocessed_train_data1/')
count=0
batch=0
for filename in os.listdir(path):
    img = cv2.imread(path+filename)

    rects = detector(img, 1)
    if (len(rects)==0):
        print("invalid: "+filename)
        invalidNames.append(filename) # just to store name of images not processed
        continue
    validNames.append(filename) # just to store name of files processed
    rect = rects[0]
    shape= predictor(img, rect)
    shape = shape_to_np(shape)
    #print(shape) #apparently coords of landmarks of face

    image_landmarks = shape[17:]
    face_rect = [image_landmarks[:,0].min(), image_landmarks[:,1].min(), image_landmarks[:,0].max(), image_landmarks[:,1].max()] #(opposite corners of rect)
    face_landmarks = image_landmarks-[image_landmarks[:,0].min(), image_landmarks[:,1].min()] # translating coords of landmarks_image to landmarks_face
    face = img[face_rect[1]:face_rect[3], face_rect[0]:face_rect[2]]

    ################################
    # align
    ##################################
    #scale face

    scale = np.random.uniform(0.75,0.95)


    # upscaling the standard aligned face preset coordinates to the size of face in input image (so input image's face can be aligned
    # similar to the standard one)
    aligned_landmarks = aligned_landmarks_norm*[face.shape[1], face.shape[0]]
    aligned_landmarks_translated = aligned_landmarks+[image_landmarks[:,0].min(), image_landmarks[:,1].min()]

    # get transformation matrix to align the face
    trans_matrix_squareshape = umeyama(image_landmarks, aligned_landmarks_translated, True) # has to be square shaped for math purposes
    trans_matrix = trans_matrix_squareshape[0:2] # but we need only 2x3 matrix, so we discard 1 dummy row

    #align face
    tempimg = img.copy()
    img_aligned = cv2.warpAffine(tempimg,trans_matrix,(tempimg.shape[1], tempimg.shape[0]), borderMode=cv2.BORDER_REPLICATE)
    rects = detector(img_aligned, 1)
    if (len(rects)==0):
        print("invalid at check 2: "+filename)
        invalidNames.append(filename) # just to store name of images not processed
        continue
    rect= rects[0]
    shape= predictor(img_aligned, rect)
    shape = shape_to_np(shape)
    image_landmarks = shape[17:]
    face_rect = [image_landmarks[:,0].min(), image_landmarks[:,1].min(), image_landmarks[:,0].max(), image_landmarks[:,1].max()] #(opposite corners of rect)

    new_top_left = (face_rect[0], face_rect[1])
    new_bottom_right = (face_rect[2], face_rect[3])


    # crop face region from original image with some random 15-25% extra area
    #################################
    h, w = img.shape[:2]
    x1, y1 = new_top_left[0], new_top_left[1]
    x2, y2 = new_bottom_right[0], new_bottom_right[1]
    delta_x = (x2 - x1) / 8
    delta_y = (y2 - y1) / 5
    delta_x = np.random.randint(delta_x, high=delta_x*1.20)
    delta_y = np.random.randint(delta_y, high=delta_y*1.20)
    x1_ = np.int(np.maximum(0, x1 - delta_x))
    x2_ = np.int(np.minimum(w-1, x2 + delta_x))
    y1_ = np.int(np.maximum(0, y1 - delta_y))
    y2_ = np.int(np.minimum(h-1, y2 + delta_y))

    ###################################

    cropped_img_aligned = img_aligned[y1_:y2_, x1_:x2_, :] # crop face region from original image with some random 15-25% extra area

    s = 240/face.shape[0]
    img_aligned_downscale = cv2.resize(img_aligned, (0,0), fx=s, fy=s) #resize to a standard size before apply random resize to get somewhat predicted ans

    img_aligned_downscale = cv2.resize(img_aligned_downscale, (0,0), fx=scale, fy=scale)

    #blur face
    img_aligned_blur = cv2.GaussianBlur(img_aligned_downscale,(5,5),0)

    img_aligned_blur = cv2.resize(img_aligned_blur, (tempimg.shape[1], tempimg.shape[0]))

    #img_new = face_blur[y1_:y2_, x1_:x2_, :]
    negImg = img_aligned.copy()
    negImg[y1:y2, x1:x2] = img_aligned_blur[y1:y2, x1:x2]
    cropped_neg_img = negImg[y1_:y2_, x1_:x2_, :]

    ###############################
    #copy pasting deepfake triangle face
    ################################
    hull = cv2.convexHull(image_landmarks-[x1_, y1_])
    mask = cv2.drawContours(np.zeros((cropped_img_aligned.shape[0],cropped_img_aligned.shape[1]), dtype="uint8"), [hull], -1, (255), -1)

    kernel = np.ones((15,15),np.uint8)

    mask1 = cv2.dilate(mask,kernel,iterations = 1)

    mask2 = cv2.bitwise_not(mask1)

    res1 = cv2.bitwise_and(cropped_neg_img, cropped_neg_img, mask=mask1)
    res2 = cv2.bitwise_and(cropped_img_aligned, cropped_img_aligned, mask=mask2)
    probability = np.random.uniform(low=0.0, high=1.0)


    if (probability<0.83): # in 83% deepfakes, triangular face region is blurred. in 20% rectangular region is blurred
        finalNegImg = res1+res2
    else:
        finalNegImg= cropped_neg_img
    negImages.append(finalNegImg)
    positiveImages.append(cropped_img_aligned)
    count=count+1
    print ("\r Images processed... {}".format(count), end="")

    if(len(negImages)%100==0):
        for index in range(len(negImages)):
            cv2.imwrite('/content/processed_train_data1/real/'+validNames[index], positiveImages[index])
            cv2.imwrite('/content/processed_train_data1/fake/'+validNames[index].replace("real", "fake"), negImages[index])
        negImages=[]
        positiveImages=[]
        invalidNames=[]
        validNames=[]
        batch = batch+1
        print("\nBatch Cleared "+str(batch)+"\n")

In [ ]:
if(len(negImages)>0):
    for index in range(len(negImages)):
        cv2.imwrite('/content/processed_train_data1/real/'+validNames[index], positiveImages[index])
        cv2.imwrite('/content/processed_train_data1/fake/'+validNames[index].replace("real", "fake"), negImages[index])
    negImages=[]
    positiveImages=[]
    invalidNames=[]
    validNames=[]

In [ ]:
!zip -r '/content/processed_train_data1zip.zip' '/content/processed_train_data1'

Streaming output truncated to the last 5000 lines.
  adding: content/processed_train_data1/real/real_10998.jpg (deflated 0%)
  adding: content/processed_train_data1/real/real_15907.jpg (deflated 0%)
  adding: content/processed_train_data1/real/real_18214.jpg (deflated 0%)
  adding: content/processed_train_data1/real/real_11283.jpg (deflated 0%)
  adding: content/processed_train_data1/real/real_11416.jpg (deflated 0%)
  adding: content/processed_train_data1/real/real_14002.jpg (deflated 0%)
  adding: content/processed_train_data1/real/real_16124.jpg (deflated 0%)
  adding: content/processed_train_data1/real/real_14918.jpg (deflated 0%)
  adding: content/processed_train_data1/real/real_13086.jpg (deflated 0%)
  adding: content/processed_train_data1/real/real_17612.jpg (deflated 0%)
  adding: content/processed_train_data1/real/real_16165.jpg (deflated 0%)
  adding: content/processed_train_data1/real/real_17174.jpg (deflated 0%)
  adding: content/processed_train_data1/real/real_9738.jpg (d

In [ ]:
!cp '/content/processed_train_data1zip.zip' '/content/gdrive/MyDrive/dfd_processed_traindata_v3_1'

# trying 2 at once

In [ ]:
if not os.path.exists('/content/unprocessed_train_data2'):
    os.mkdir('/content/unprocessed_train_data2')

if not os.path.exists('/content/processed_train_data2'):
    os.mkdir('/content/processed_train_data2')

if not os.path.exists('/content/processed_train_data2/real'):
    os.mkdir('/content/processed_train_data2/real')

if not os.path.exists('/content/processed_train_data2/fake'):
    os.mkdir('/content/processed_train_data2/fake')

In [ ]:
!unzip '/content/gdrive/MyDrive/unprocessed_trainset_2.zip' -d '/content/unprocessed_train_data2'

Streaming output truncated to the last 5000 lines.
  inflating: /content/unprocessed_train_data2/real_22889.jpg  
  inflating: /content/unprocessed_train_data2/real_22851.jpg  
  inflating: /content/unprocessed_train_data2/real_22768.jpg  
  inflating: /content/unprocessed_train_data2/real_28118.jpg  
  inflating: /content/unprocessed_train_data2/real_20248.jpg  
  inflating: /content/unprocessed_train_data2/real_21936.jpg  
  inflating: /content/unprocessed_train_data2/real_23125.jpg  
  inflating: /content/unprocessed_train_data2/real_26029.jpg  
  inflating: /content/unprocessed_train_data2/real_28645.jpg  
  inflating: /content/unprocessed_train_data2/real_22481.jpg  
  inflating: /content/unprocessed_train_data2/real_27813.jpg  
  inflating: /content/unprocessed_train_data2/real_21273.jpg  
  inflating: /content/unprocessed_train_data2/real_21802.jpg  
  inflating: /content/unprocessed_train_data2/real_20730.jpg  
  inflating: /content/unprocessed_train_data2/real_26079.jpg  
  in

In [ ]:
negImages=[]
positiveImages=[]
invalidNames=[]
validNames=[]
path = ('/content/unprocessed_train_data2/')
count=0
batch=0
for filename in os.listdir(path):
    img = cv2.imread(path+filename)

    rects = detector(img, 1)
    if (len(rects)==0):
        print("invalid: "+filename)
        invalidNames.append(filename) # just to store name of images not processed
        continue
    validNames.append(filename) # just to store name of files processed
    rect = rects[0]
    shape= predictor(img, rect)
    shape = shape_to_np(shape)
    #print(shape) #apparently coords of landmarks of face

    image_landmarks = shape[17:]
    face_rect = [image_landmarks[:,0].min(), image_landmarks[:,1].min(), image_landmarks[:,0].max(), image_landmarks[:,1].max()] #(opposite corners of rect)
    face_landmarks = image_landmarks-[image_landmarks[:,0].min(), image_landmarks[:,1].min()] # translating coords of landmarks_image to landmarks_face
    face = img[face_rect[1]:face_rect[3], face_rect[0]:face_rect[2]]

    ################################
    # align
    ##################################
    #scale face

    scale = np.random.uniform(0.75,0.95)


    # upscaling the standard aligned face preset coordinates to the size of face in input image (so input image's face can be aligned
    # similar to the standard one)
    aligned_landmarks = aligned_landmarks_norm*[face.shape[1], face.shape[0]]
    aligned_landmarks_translated = aligned_landmarks+[image_landmarks[:,0].min(), image_landmarks[:,1].min()]

    # get transformation matrix to align the face
    trans_matrix_squareshape = umeyama(image_landmarks, aligned_landmarks_translated, True) # has to be square shaped for math purposes
    trans_matrix = trans_matrix_squareshape[0:2] # but we need only 2x3 matrix, so we discard 1 dummy row

    #align face
    tempimg = img.copy()
    img_aligned = cv2.warpAffine(tempimg,trans_matrix,(tempimg.shape[1], tempimg.shape[0]), borderMode=cv2.BORDER_REPLICATE)
    rects = detector(img_aligned, 1)
    if (len(rects)==0):
        print("invalid at check 2: "+filename)
        invalidNames.append(filename) # just to store name of images not processed
        continue
    rect= rects[0]
    shape= predictor(img_aligned, rect)
    shape = shape_to_np(shape)
    image_landmarks = shape[17:]
    face_rect = [image_landmarks[:,0].min(), image_landmarks[:,1].min(), image_landmarks[:,0].max(), image_landmarks[:,1].max()] #(opposite corners of rect)

    new_top_left = (face_rect[0], face_rect[1])
    new_bottom_right = (face_rect[2], face_rect[3])


    # crop face region from original image with some random 15-25% extra area
    #################################
    h, w = img.shape[:2]
    x1, y1 = new_top_left[0], new_top_left[1]
    x2, y2 = new_bottom_right[0], new_bottom_right[1]
    delta_x = (x2 - x1) / 8
    delta_y = (y2 - y1) / 5
    delta_x = np.random.randint(delta_x, high=delta_x*1.20)
    delta_y = np.random.randint(delta_y, high=delta_y*1.20)
    x1_ = np.int(np.maximum(0, x1 - delta_x))
    x2_ = np.int(np.minimum(w-1, x2 + delta_x))
    y1_ = np.int(np.maximum(0, y1 - delta_y))
    y2_ = np.int(np.minimum(h-1, y2 + delta_y))

    ###################################

    cropped_img_aligned = img_aligned[y1_:y2_, x1_:x2_, :] # crop face region from original image with some random 15-25% extra area

    s = 240/face.shape[0]
    img_aligned_downscale = cv2.resize(img_aligned, (0,0), fx=s, fy=s) #resize to a standard size before apply random resize to get somewhat predicted ans

    img_aligned_downscale = cv2.resize(img_aligned_downscale, (0,0), fx=scale, fy=scale)

    #blur face
    img_aligned_blur = cv2.GaussianBlur(img_aligned_downscale,(5,5),0)

    img_aligned_blur = cv2.resize(img_aligned_blur, (tempimg.shape[1], tempimg.shape[0]))

    #img_new = face_blur[y1_:y2_, x1_:x2_, :]
    negImg = img_aligned.copy()
    negImg[y1:y2, x1:x2] = img_aligned_blur[y1:y2, x1:x2]
    cropped_neg_img = negImg[y1_:y2_, x1_:x2_, :]

    ###############################
    #copy pasting deepfake triangle face
    ################################
    hull = cv2.convexHull(image_landmarks-[x1_, y1_])
    mask = cv2.drawContours(np.zeros((cropped_img_aligned.shape[0],cropped_img_aligned.shape[1]), dtype="uint8"), [hull], -1, (255), -1)

    kernel = np.ones((15,15),np.uint8)

    mask1 = cv2.dilate(mask,kernel,iterations = 1)

    mask2 = cv2.bitwise_not(mask1)

    res1 = cv2.bitwise_and(cropped_neg_img, cropped_neg_img, mask=mask1)
    res2 = cv2.bitwise_and(cropped_img_aligned, cropped_img_aligned, mask=mask2)
    probability = np.random.uniform(low=0.0, high=1.0)


    if (probability<0.83): # in 83% deepfakes, triangular face region is blurred. in 20% rectangular region is blurred
        finalNegImg = res1+res2
    else:
        finalNegImg= cropped_neg_img
    negImages.append(finalNegImg)
    positiveImages.append(cropped_img_aligned)
    count=count+1
    print ("\r Images processed... {}".format(count), end="")

    if(len(negImages)%100==0):
        for index in range(len(negImages)):
            cv2.imwrite('/content/processed_train_data2/real/'+validNames[index], positiveImages[index])
            cv2.imwrite('/content/processed_train_data2/fake/'+validNames[index].replace("real", "fake"), negImages[index])
        negImages=[]
        positiveImages=[]
        invalidNames=[]
        validNames=[]
        batch = batch+1
        print("\nBatch Cleared "+str(batch)+"\n")

 Images processed... 100
Batch Cleared 1

 Images processed... 200
Batch Cleared 2

 Images processed... 300
Batch Cleared 3

 Images processed... 400
Batch Cleared 4

 Images processed... 500
Batch Cleared 5

 Images processed... 600
Batch Cleared 6

 Images processed... 700
Batch Cleared 7

 Images processed... 800
Batch Cleared 8

 Images processed... 900
Batch Cleared 9

 Images processed... 1000
Batch Cleared 10

 Images processed... 1100
Batch Cleared 11

 Images processed... 1195invalid at check 2: real_19703.jpg
 Images processed... 1200
Batch Cleared 12

 Images processed... 1300
Batch Cleared 13

 Images processed... 1400
Batch Cleared 14

 Images processed... 1500
Batch Cleared 15

 Images processed... 1600
Batch Cleared 16

 Images processed... 1700
Batch Cleared 17

 Images processed... 1800
Batch Cleared 18

 Images processed... 1900
Batch Cleared 19

 Images processed... 2000
Batch Cleared 20

 Images processed... 2100
Batch Cleared 21

 Images processed... 2200
Batch Cl

In [ ]:
if(len(negImages)>0):
    for index in range(len(negImages)):
        cv2.imwrite('/content/processed_train_data2/real/'+validNames[index], positiveImages[index])
        cv2.imwrite('/content/processed_train_data2/fake/'+validNames[index].replace("real", "fake"), negImages[index])
    negImages=[]
    positiveImages=[]
    invalidNames=[]
    validNames=[]

In [ ]:
!zip -r '/content/processed_train_data2zip.zip' '/content/processed_train_data2'

Streaming output truncated to the last 5000 lines.
  adding: content/processed_train_data2/real/real_25049.jpg (deflated 0%)
  adding: content/processed_train_data2/real/real_22351.jpg (deflated 0%)
  adding: content/processed_train_data2/real/real_26902.jpg (deflated 0%)
  adding: content/processed_train_data2/real/real_28018.jpg (deflated 0%)
  adding: content/processed_train_data2/real/real_28083.jpg (deflated 0%)
  adding: content/processed_train_data2/real/real_25656.jpg (deflated 0%)
  adding: content/processed_train_data2/real/real_20940.jpg (deflated 0%)
  adding: content/processed_train_data2/real/real_25927.jpg (deflated 0%)
  adding: content/processed_train_data2/real/real_25624.jpg (deflated 0%)
  adding: content/processed_train_data2/real/real_22690.jpg (deflated 0%)
  adding: content/processed_train_data2/real/real_22188.jpg (deflated 0%)
  adding: content/processed_train_data2/real/real_26063.jpg (deflated 0%)
  adding: content/processed_train_data2/real/real_27442.jpg (

In [ ]:
!cp '/content/processed_train_data2zip.zip' '/content/gdrive/MyDrive/dfd_processed_traindata_v3_2'